In [ ]:

!pip install tensorflow opencv-python-headless numpy matplotlib scikit-image

import os
import zipfile
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tensorflow as tf

from skimage.color import rgb2lab, lab2rgb

from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam


from google.colab import drive
drive.mount('/content/drive')


drive_path_prefix = '/content/drive/MyDrive/'
train_zip_path = os.path.join(drive_path_prefix, 'train2017.zip')
val_zip_path = os.path.join(drive_path_prefix, 'val2017.zip')

train_dir = 'train2017'
val_dir = 'val2017'




os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)


if not os.listdir(train_dir):
    print(f"'{train_dir}' klasörü boş. Zipten çıkarma işlemi başlatılıyor...")
    with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
        zip_ref.extractall(train_dir)
    print("Eğitim verileri başarıyla çıkartıldı.")
else:
    print("Eğitim verileri zaten mevcut, çıkartma işlemi atlandı.")


if not os.listdir(val_dir):
    print(f"'{val_dir}' klasörü boş. Zipten çıkarma işlemi başlatılıyor...")
    with zipfile.ZipFile(val_zip_path, 'r') as zip_ref:
        zip_ref.extractall(val_dir)
    print("Doğrulama verileri başarıyla çıkartıldı.")
else:
    print("Doğrulama verileri zaten mevcut, çıkartma işlemi atlandı.")

Mounted at /content/drive
'train2017' klasörü boş. Zipten çıkarma işlemi başlatılıyor...
Eğitim verileri başarıyla çıkartıldı.
'val2017' klasörü boş. Zipten çıkarma işlemi başlatılıyor...
Doğrulama verileri başarıyla çıkartıldı.


In [ ]:

IMG_SIZE = 256
BATCH_SIZE = 32 


actual_train_path = os.path.join(train_dir, 'train2017')
actual_val_path = os.path.join(val_dir, 'val2017')


train_paths = [os.path.join(actual_train_path, fname) for fname in os.listdir(actual_train_path)]
val_paths = [os.path.join(actual_val_path, fname) for fname in os.listdir(actual_val_path)]

print(f"Gerçek eğitim resim yolu: '{actual_train_path}'")
print(f"Toplam eğitim resmi sayısı: {len(train_paths)}")
print(f"Gerçek doğrulama resim yolu: '{actual_val_path}'")
print(f"Toplam doğrulama resmi sayısı: {len(val_paths)}")



def parse_image(filename):
    """
    Bir dosya yolunu alır, resmi okur, işler ve model için
    girdi (L kanalı) ve çıktı (ab kanalları) tensörlerine dönüştürür.
    """
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])

    def to_lab(rgb_image):
        lab = rgb2lab(rgb_image.numpy())
        L = lab[:, :, 0] / 50.0 - 1.0
        ab = lab[:, :, 1:] / 128.0
        return L.reshape(IMG_SIZE, IMG_SIZE, 1).astype(np.float32), ab.astype(np.float32)

    L, ab = tf.py_function(to_lab, [image], [tf.float32, tf.float32])

    L.set_shape([IMG_SIZE, IMG_SIZE, 1])
    ab.set_shape([IMG_SIZE, IMG_SIZE, 2])

    return L, ab


AUTOTUNE = tf.data.AUTOTUNE

def create_dataset(paths):
    dataset = tf.data.Dataset.from_tensor_slices(paths)
    dataset = dataset.map(parse_image, num_parallel_calls=AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    return dataset


train_dataset = create_dataset(train_paths)
val_dataset = create_dataset(val_paths)


for L_batch, ab_batch in train_dataset.take(1): 
    print("Girdi (L) batch şekli:", L_batch.shape)
    print("Çıktı (ab) batch şekli:", ab_batch.shape)

Gerçek eğitim resim yolu: 'train2017/train2017'
Toplam eğitim resmi sayısı: 118287
Gerçek doğrulama resim yolu: 'val2017/val2017'
Toplam doğrulama resmi sayısı: 5000
Girdi (L) batch şekli: (32, 256, 256, 1)
Çıktı (ab) batch şekli: (32, 256, 256, 2)


In [ ]:

def build_unet_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    u4 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u4 = Concatenate()([u4, c4])
    u4 = Conv2D(512, (3, 3), activation='relu', padding='same')(u4)
    u4 = Conv2D(512, (3, 3), activation='relu', padding='same')(u4)
    u3 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(u4)
    u3 = Concatenate()([u3, c3])
    u3 = Conv2D(256, (3, 3), activation='relu', padding='same')(u3)
    u3 = Conv2D(256, (3, 3), activation='relu', padding='same')(u3)
    u2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(u3)
    u2 = Concatenate()([u2, c2])
    u2 = Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    u2 = Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    u1 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u2)
    u1 = Concatenate()([u1, c1])
    u1 = Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    u1 = Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    outputs = Conv2D(2, (1, 1), activation='tanh', padding='same')(u1)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model


model = build_unet_model(input_shape=(IMG_SIZE, IMG_SIZE, 1))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')
model.summary()


drive_save_path = '/content/drive/MyDrive/Nostalji_Makinesi_Projesi/'
os.makedirs(drive_save_path, exist_ok=True)
checkpoint_path = os.path.join(drive_save_path, "colorization_unet_generator_best.h5")

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)


history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[model_checkpoint_callback, early_stopping_callback]
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 16, 16,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 16, 16,    │  4,719,616 │ max_pooling2d_3[… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 16, 16,    │  9,438,208 │ conv2d_8[0][0]    │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 32, 32,    │  2,097,664 │ conv2d_9[0][0]    │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ conv2d_transpose

 Total params: 31,030,658 (118.37 MB)

 Trainable params: 31,030,658 (118.37 MB)

 Non-trainable params: 0 (0.00 B)

...

3697/3697 ━━━━━━━━━━━━━━━━━━━━ 990s 267ms/step - loss: 0.0105 - val_loss: 0.0102
Epoch 5/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0102
Epoch 5: val_loss improved from 0.01021 to 0.01013, saving model to /content/drive/MyDrive/Nostalji_Makinesi_Projesi/colorization_unet_generator_best.h5


3697/3697 ━━━━━━━━━━━━━━━━━━━━ 989s 267ms/step - loss: 0.0102 - val_loss: 0.0101
Epoch 6/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0100
Epoch 6: val_loss did not improve from 0.01013
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 983s 265ms/step - loss: 0.0100 - val_loss: 0.0103
Epoch 7/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - loss: 0.0098
Epoch 7: val_loss improved from 0.01013 to 0.00989, saving model to /content/drive/MyDrive/Nostalji_Makinesi_Projesi/colorization_unet_generator_best.h5


3697/3697 ━━━━━━━━━━━━━━━━━━━━ 992s 268ms/step - loss: 0.0098 - val_loss: 0.0099
Epoch 8/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - loss: 0.0096
Epoch 8: val_loss improved from 0.00989 to 0.00981, saving model to /content/drive/MyDrive/Nostalji_Makinesi_Projesi/colorization_unet_generator_best.h5


3697/3697 ━━━━━━━━━━━━━━━━━━━━ 994s 268ms/step - loss: 0.0096 - val_loss: 0.0098
Epoch 9/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0095
Epoch 9: val_loss did not improve from 0.00981
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 984s 265ms/step - loss: 0.0095 - val_loss: 0.0099
Epoch 10/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0093
Epoch 10: val_loss improved from 0.00981 to 0.00963, saving model to /content/drive/MyDrive/Nostalji_Makinesi_Projesi/colorization_unet_generator_best.h5


3697/3697 ━━━━━━━━━━━━━━━━━━━━ 989s 267ms/step - loss: 0.0093 - val_loss: 0.0096
Epoch 11/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0090
Epoch 11: val_loss did not improve from 0.00963
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 983s 265ms/step - loss: 0.0090 - val_loss: 0.0097
Epoch 12/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0087
Epoch 12: val_loss did not improve from 0.00963
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 982s 265ms/step - loss: 0.0087 - val_loss: 0.0099
Epoch 13/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0083
Epoch 13: val_loss did not improve from 0.00963
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 983s 265ms/step - loss: 0.0083 - val_loss: 0.0104
Epoch 14/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0079
Epoch 14: val_loss did not improve from 0.00963
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 984s 265ms/step - loss: 0.0079 - val_loss: 0.0101
Epoch 15/50
3697/3697 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - loss: 0.0074
Epoch 15: val_loss did not improve from 0.0

In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kayıp Grafiği')
plt.xlabel('Epoch')
plt.ylabel('Kayıp (MSE)')
plt.legend()
plt.grid(True)
plt.show()


loaded_model = tf.keras.models.load_model(checkpoint_path)


for L_batch, ab_batch in val_dataset.take(1):
   
    predicted_ab_batch = loaded_model.predict(L_batch)

    
    for i in range(min(5, BATCH_SIZE)):
        
        L_input = L_batch[i].numpy()
        ab_ground_truth = ab_batch[i].numpy()
        predicted_ab = predicted_ab_batch[i]

        
        original_lab = np.concatenate([((L_input + 1) * 50.0), (ab_ground_truth * 128.0)], axis=-1)
        original_rgb = lab2rgb(original_lab)

        
        predicted_lab = np.concatenate([((L_input + 1) * 50.0), (predicted_ab * 128.0)], axis=-1)
        predicted_rgb = lab2rgb(predicted_lab)

        
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.title("Gri Tonlamalı Girdi")
        plt.imshow(L_input.squeeze(), cmap='gray')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.title("Modelin Çıktısı")
        plt.imshow(np.clip(predicted_rgb, 0, 1)) 
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.title("Orijinal Renkli")
        plt.imshow(np.clip(original_rgb, 0, 1))
        plt.axis('off')
        plt.show()